In [155]:
import requests as req
import numpy as np
import json
import random
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [156]:
class InterestsParser:
    def __init__(self, trees):
        self.trees = trees
        self.count = 0
        
        for tree in trees:
            self.count += _count_interests(tree)
            
    def parse(self, interests_list, prop):
        interests_vec = np.zeros(self.count, np.float)
        interests_set = frozenset(interests_list)
        i_p = [0]
        weight = 1 / len(self.trees)

        for tree in self.trees:
            _parse_interests(interests_set, tree, interests_vec, i_p, weight, prop)
        
        return interests_vec

def _count_interests(node):
    count = 1
    if 'children' in node:
        for c in node['children']:
            count = count + _count_interests(c)

    return count

def _parse_interests(i_set, i_node, i_vec, i_p, weight, prop):
    val = 0

    if 'children' in i_node:
        children = i_node['children']
        n_children = len(children)
        for c in children:
            val += _parse_interests(i_set, c, i_vec, i_p, 1 / n_children, prop)

    if prop in i_node:
        if i_node[prop] in i_set:
            val = 1

    i_vec[i_p[0]] = val * weight
    i_p[0] += 1

    return val

In [157]:
class Person:
    def __init__(self, p_id, interests):
        self.p_id = p_id
        self.interests = interests
        
    def __str__(self):
        return 'id: {0} interests: {1}'.format(self.p_id, self.interests)
    
    def __repr__(self):
        return self.__str__()
    
def parse_person(p, interest_parser):
    i = 0
    num = ''
    if len(p) < 1:
        return None

    while p[i] != ' ':
        num = num + p[i]
        i += 1

    p_id = int(num)

    while p[i] != '[':
        i += 1

    s = i + 1

    while p[i] != ']':
        i += 1

    f = i

    interests = interest_parser.parse(p[s:f].split(','), 'name')

    return Person(p_id, interests)

In [158]:
class Topic:
    def __init__(self, t_id, name, interest, popularity, penalty):
        self.t_id = t_id
        self.name = name
        self.interest = interest
        self.popularity = popularity
        self.penalty = penalty
        
    def __str__(self):
        return 'id: {0} interest: {1}'.format(self.t_id, self.interest)
    
    def __repr__(self):
        return self.__str__()
    
def parse_topics(topic_ranks, topic_interests):
    topics = {}
    t_id = 0
    no_match = 0

    for t, r in topic_ranks.items():
        topics[t] = Topic(t_id, t, None, r, 0)
        t_id += 1

    for t in topic_interests:
        kw = t['keyword']
        if kw in topics:
            topics[kw].interest = interest_parser.parse(t['tags'], 'id')
        else:
            no_match += 1

    topics = list(filter(lambda x: x.interest is not None, topics.values()))
    
    print('Ranked: {}'.format(len(topic_ranks)))
    print('Classified: {}'.format(len(topic_interests)))
    print('Unique: {}'.format(len(topics)))
    print('No match: {}'.format(no_match))
    
    return topics

In [159]:
class AvalinguoRoom:
    def __init__(self):
        self.users = list()
        self._interests = None
    
    def addUser(self, pers):
        if self._interests is None:
            shape = np.shape(pers.interests)
            self._interests = np.zeros(shape)
        
        self.users.append(pers)
        self._interests += pers.interests
        
    @property
    def interests(self):
        return self._interests / len(self.users)

In [161]:
def comb(dist, pop):
    n = len(dist)
    dist = np.reshape(dist, (n, 1))
    pop = np.reshape(pop, (n, 1))
    tmp = np.concatenate((dist, pop), axis=1
                        )
    W = np.array([[1.5], [2]])
    
    return tmp * W

# def reccomend(group, topics, n):
#  using old version of recommend function
def reccomend(group, topics, n):
    ints = np.array([t.interest for t in topics])
    diff = ints - group.interests
    dist = np.sqrt(np.sum(diff**2,1))
    
    rec = np.argpartition(dist, -n)[-n:]
    return rec, dist[rec]

In [160]:
with open('interests-tree-nested.json') as tree:
    interest_parser = InterestsParser(json.loads(tree.read()))

with open('synthtetic_dataset.lg') as dataset_file:
    raw_data = dataset_file.read()
    
with open('normalizedRanks.json') as topic_ranks_json:
    topic_ranks = json.loads(topic_ranks_json.read())

with open('first.json') as topic_interests_json:
    topic_interests = json.loads(topic_interests_json.read())

lines = raw_data.split('\n')

headers = lines[0:4]
body = lines[5:-1]

In [162]:
persons = [parse_person(p, interest_parser) for p in body]
topics = parse_topics(topic_ranks, topic_interests)

Ranked: 474
Classified: 123
Unique: 89
No match: 12


In [163]:
def distance(v1, v2):
    if len(np.shape(v1)) > 1: 
        return np.sqrt(np.sum((v1 - v2) ** 2, 1))
    return np.sqrt(np.sum((v1 - v2) ** 2))

def make_rooms(n_rooms, room_size, persons):
    rooms = [AvalinguoRoom() for i in range(n_rooms)]
    interests= np.array([p.interests for p in persons])
    labels = KMeans(n_clusters=n_rooms, random_state=0).fit_predict(interests)
    
    for p, c in zip(persons, labels):
        if len(rooms[c].users) < room_size:
            rooms[c].addUser(p)
    return rooms

def experiment(rooms, topics):
    avg_interests = np.mean(interests, 0)
    
    d_group = np.zeros(n_rooms)
    d_int_mean = np.zeros(n_rooms)
    d_int_std = np.zeros(n_rooms)
    
#     getting recommendations and calculating mean distance
    for r, room in enumerate(rooms):        
        rec, dist = reccomend(room, topics, 5)
        
        rec_vec = np.array([topics[r].name for r in rec])
        
        mean_rec = np.mean(rec_vec, 0)
        users_matrix = np.array([p.interests for p in room.users])

        d_interest = distance(users_matrix, mean_vec)
        
        d_group[r] = distance(room.interests, avg_interests)
        d_int_mean[r] = np.mean(d_interest)
        d_int_std[r] = np.std(d_interest)

#     for room in rooms:

    
    
    
#     distances
#     mean_distance
    
    return(d_group, d_int_mean, d_int_std)

In [165]:
counter = {}
for room in rooms:
#     print([u.p_id for u in room.users])
    rec, dist = reccomend(room, topics, 10)
    
    for i in range(len(rec)):
        rec_name = topic_interests[topics[rec[i]].t_id]['keyword']
        
        if rec_name not in counter:
            counter[rec_name] = 0
        
        counter[rec_name] += 1
        
#         print(rec_name, dist[i])
        
    

In [164]:
# a = np.array([1, 2])
# b = np.concatenate((a, a), 1)

In [166]:
group, mean, std = experiment(1000, 4, persons, topics)

TypeError: cannot perform reduce with flexible type

In [ ]:
#plt.plot(group[0:8], mean[0:8], ".")
#plt.plot(group[0:8], (mean - std)[0:8], "^")
#plt.plot(group[0:8], (mean + std)[0:8], "+")